# Random graph experiments I: Color

## Graph and data generation module ##

In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "64" # export OMP_NUM_THREADS=4
os.environ["OPENBLAS_NUM_THREADS"] = "64" # export OPENBLAS_NUM_THREADS=4 
os.environ["VECLIB_MAXIMUM_THREADS"] = "64" # export VECLIB_MAXIMUM_THREADS=4

In [2]:
import sys, time, random
import numpy as np
from collections import Counter, defaultdict, deque
from datetime import datetime

In [3]:
def flatten(l):
  return deque([i for j in l for i in j])

In [4]:
"""
  Define random_integer_excluding
  
    Params:
      low, high: integers indicating the range for sampling. Inclusive low and exclusive high.
      excluding: a set
  
    Functionality:
      Extend random.sample function to sample from large integer range, and exclude a small set
      Optimized when the range (high-low) is very large
      
    Return:
      sampled_integers (list)
"""
def random_integer_excluding(low, high, excluding, size=1):
  exclusion = {num for num in excluding if low <= num < high}
  result = []
  if size == 0:
    return result
  while len(result) < size:
    buf = random.sample(range(low, high), size - len(result))
    for num in buf:
      if len(exclusion) >= high-low:
        raise RuntimeError("random_integer_excluding fails: Size too large; exhausted all integers available in the range before finishing.")
      if num not in exclusion:
        result.append(num)
        exclusion.add(num)
  return result

In [5]:
def collect(assignment):
  result = defaultdict(set)
  for node in assignment:
    result[assignment[node]].add(node)
  return result

In [6]:
"""
  Define stratification_indexer
  
    Params: 
    - groups (an map from group_id to list of member)
      groups[i] return members of group i.
      Group 0 is the 'default' group and is not present.
      Can be negative, which will always be ignored.
    - strata_sizes (an integer array of length = number of strata)
      strata_sizes[i] return the size cap of the stratum i (except i=0)
    - num_multisample (an integer)
    
    Functionality:
      Perform sampling on the same set of parameters num_multisample time.
      In general case, sample strata_sizes[i] nodes from group i w/o replacement.
      
      If the specified strata_size[i] is larger than the number of nodes in group i,
      retuen all nodes in group i, and increase the number of sample in the default
      strata (group 0) with strata_size[i] - actual size of group[i].
      
    Return:
      sampled_nodes, (numpy.ndarray)
      log_down_sampling_rate for each sampled node, (numpy.ndarray)
"""
def stratification_indexer(num_nodes, groups_assignment, strata_sizes, post_assignments={}, num_multisample=1):
  
  num_strata = len(strata_sizes)
  groups = collect(groups_assignment)
  sizeof_pool0 = num_nodes - sum([len(groups[i]) for i in groups])
  
  def sample_once():
    
    sizeof_strata0 = strata_sizes[0]
    selected_indices_for_strata = [None] * num_strata
    
    for i in range(1, num_strata):
      selected_indices_for_strata[i] = random.sample(groups[i], min(len(groups[i]), strata_sizes[i]))
      sizeof_strata0 += strata_sizes[i] - len(selected_indices_for_strata[i])
    
    selected_indices_for_strata[0] = random_integer_excluding(low=0, high=num_nodes, excluding=groups_assignment, size=sizeof_strata0)
    
    if (sizeof_strata0 == 0) ^ (sizeof_pool0 == 0):
      raise RuntimeError("Size of strata and pool must be both non-zero or both zero. (Currently {}, {})".format(sizeof_strata0, sizeof_pool0))
    actual_strata_sizes = [sizeof_strata0]
    pool_sizes = [sizeof_pool0]
    for i in range(1, num_strata):
      if len(selected_indices_for_strata[i]) == 0 ^ len(groups[i]) == 0:
        raise RuntimeError("Size of strata and pool must be both non-zero or both zero.")
        
      actual_strata_sizes.append(len(selected_indices_for_strata[i]))
      pool_sizes.append(len(groups[i]))
 
    for n in post_assignments:
      actual_strata_sizes[post_assignments[n]] += 1
      pool_sizes[post_assignments[n]] += 1
      
    
    nested_strata_sizes = [[actual_strata_sizes[i]] * len(selected_indices_for_strata[i]) for i in range(num_strata)]
    nested_pool_sizes = [[pool_sizes[i]] * len(selected_indices_for_strata[i]) for i in range(num_strata)]
    
    for n in post_assignments:
      selected_indices_for_strata.append([n])
      nested_strata_sizes.append([actual_strata_sizes[post_assignments[n]]])
      nested_pool_sizes.append([pool_sizes[post_assignments[n]]])
    
    return flatten(selected_indices_for_strata), flatten(nested_strata_sizes), flatten(nested_pool_sizes)
  
  return [sample_once() for _ in range(num_multisample)]

In [7]:
def test():
  groups_assignment = {0:-1, 1:-1, 2:1, 3:1, 4:1, 5:2, 6:2}
  strata_sizes = np.array([0,2,1])
  return stratification_indexer(7, groups_assignment, strata_sizes, post_assignments={1:1}, num_multisample=2)
test()

[(deque([4, 3, 5, 1]), deque([3, 3, 1, 3]), deque([4, 4, 2, 4])),
 (deque([3, 2, 5, 1]), deque([3, 3, 1, 3]), deque([4, 4, 2, 4]))]

In [8]:
def test():
  groups_assignment = {0:-1, 1:-1, 2:1, 3:1, 4:1, 5:2, 6:2}
  strata_sizes = np.array([1,2,3])
  return stratification_indexer(9, groups_assignment, strata_sizes, post_assignments={1:2}, num_multisample=2)
test()

[(deque([7, 8, 2, 3, 6, 5, 1]),
  deque([2, 2, 2, 2, 3, 3, 3]),
  deque([2, 2, 3, 3, 3, 3, 3])),
 (deque([8, 7, 2, 4, 5, 6, 1]),
  deque([2, 2, 2, 2, 3, 3, 3]),
  deque([2, 2, 3, 3, 3, 3, 3]))]

In [9]:
class DirectedColorGraphFeatureExtractor(object):
  
  def __init__(self, num_nodes, num_multisample, graph_info=None):
    self.num_nodes = num_nodes
    self.adjacency_to = [Counter() for _ in range(num_nodes)]
    self.adjacency_from = [Counter() for _ in range(num_nodes)]
    self.in_degs = np.zeros(num_nodes)
    self.out_degs = np.zeros(num_nodes)
    self.graph_info = graph_info
    self.num_multisample = num_multisample
    self.data = {i:[[] for _ in range(num_multisample)] for i in ['Xs_u', 'ys_u', 'sws_u',
                                                                  'Xs_s', 'ys_s', 'sws_s',
                                                                  'Xs_i', 'ys_i', 'sws_i']}
    
  def add_edge(self, actor, target, warn=True):
    if self.adjacency_to[actor][target] == 0:
      self.out_degs[actor] += 1
      self.in_degs[target] += 1
      self.adjacency_to[actor][target] += 1
      self.adjacency_from[target][actor] += 1
    elif warn:
      print("Edge already exists; ignored.")
      
  
  def uniform_sample(self, actor, target, num_multisample=1):
    # Exclude actor, target, those already connected
    assignment = {}
    for friend in self.adjacency_to[actor]:
      assignment[friend] = -1
    assignment[actor] = -1
    assignment[target] = -1
    return stratification_indexer(self.num_nodes, assignment, [99000], post_assignments={target:0}, num_multisample=num_multisample)
  
    
  def stratified_sample(self, actor, target, num_multisample=1):
    assignment = {}
    for node in self.graph_info['red_nodes']:
      assignment[node] = 1
    for friend in self.adjacency_to[actor]:
      assignment[friend] = -1
    assignment[actor] = -1
    assignment[target] = -1
    target_group = int(self.graph_info['node_colors'][target])
    return stratification_indexer(self.num_nodes, assignment, [12,12], post_assignments={target:target_group}, num_multisample=num_multisample) 
    
  def importance_sample(self, actor, target, num_multisample=1):
    assignment = {}
    for node in self.graph_info['red_nodes']:
      assignment[node] = 1
    for friend in self.adjacency_to[actor]:
      assignment[friend] = -1
    assignment[actor] = -1
    assignment[target] = -1
    target_group = int(self.graph_info['node_colors'][target])
    return stratification_indexer(self.num_nodes, assignment, [3,21], post_assignments={target:target_group}, num_multisample=num_multisample)
    
  def extract_feature(self, actor, candidates):
    #in_degs = self.in_degs[candidates]
    #log_in_degs = np.log(in_degs + (in_degs < 0.5).astype(int))
    return np.array([self.graph_info['node_colors'][candidates], self.graph_info['node_fitness'][candidates]]).T
  
  def add_edge_and_collect_data(self, actor, target):
    for i, (candidates, strata, pool) in enumerate(self.uniform_sample(actor, target, num_multisample=self.num_multisample)):
      self.data['Xs_u'][i].append(self.extract_feature(actor, candidates))
      self.data['ys_u'][i].append(99000)
      self.data['sws_u'][i].append(np.log(np.array(pool)/np.array(strata)))
    #for i, (candidates, strata, pool) in enumerate(self.stratified_sample(actor, target, num_multisample=self.num_multisample)):
    #  self.data['Xs_s'][i].append(self.extract_feature(actor, candidates))
    #  self.data['ys_s'][i].append(24)
    #  self.data['sws_s'][i].append(np.log(np.array(pool)/np.array(strata)))  
    #for i, (candidates, strata, pool) in enumerate(self.importance_sample(actor, target, num_multisample=self.num_multisample)):
    #  self.data['Xs_i'][i].append(self.extract_feature(actor, candidates))
    #  self.data['ys_i'][i].append(24)
    #  self.data['sws_i'][i].append(np.log(np.array(pool)/np.array(strata)))  
    #self.add_edge(actor, target)

In [10]:
def extract(filename):
  
  num_nodes = 100000
  num_choice_edges = 20000
  logging_interval = 100
  
  dat = np.load('graphs-fitness/' + filename)
  graph_info = {'node_colors':dat['node_colors'],\
                'node_fitness':dat['node_fitness'],\
                'red_nodes':np.argwhere(dat['node_colors']).reshape(-1)}
  G = DirectedColorGraphFeatureExtractor(num_nodes, num_multisample=1, graph_info=graph_info)
  for vi,vj in dat['er_edges']:
    G.add_edge(vi,vj)
  t0 = time.time()
  
  for i, (actor, target) in enumerate(dat['choice_edges']):
    G.add_edge_and_collect_data(actor, target)
    
    if (i+1) % logging_interval == 0 and i > 0:
      t = time.time() - t0
      t_done = datetime.fromtimestamp(t0 + t * (num_choice_edges) / (i+1)).strftime('%Y-%m-%dT%H:%M:%S') 
      tentative = "features-fitness/{}".format('&'.join(filename.split('&')[:4]))
      msg = "Time = {}; Progress = {:.2f}; Time since beginning = {:.1f}s; Est finish = {}; Tentative Destination = {}\n"\
            .format(datetime.now().strftime('%Y-%m-%dT%H:%M:%S'), (i+1)/num_choice_edges, t, t_done, tentative)
      with open("log", "a") as log_file:
        log_file.write(msg)
    
  new_filename = "features-fitness/FULL&{}&extract_time={}.npz".format('&'.join(filename.split('&')), datetime.now().strftime('%Y-%m-%dT%H:%M:%S'))
  np.savez(new_filename, Xs_u=G.data['Xs_u'], ys_u=G.data['ys_u'], sws_u=G.data['sws_u'],\
                         Xs_s=G.data['Xs_s'], ys_s=G.data['ys_s'], sws_s=G.data['sws_s'],\
                         Xs_i=G.data['Xs_i'], ys_i=G.data['ys_i'], sws_i=G.data['sws_i'])

In [11]:
import os
from multiprocessing import Pool

filenames = sorted([os.path.basename(name) for name in os.listdir('graphs-fitness') if len(name.split('&')) >= 4])
filenames = filenames[::20]

In [12]:
with Pool(8) as p:
  p.map(extract, filenames)